In [1]:
import pandas as pd
import numpy as np
import os

print("Package is ready.")

Package is ready.


## Data classifying

In [2]:
raw_df = pd.read_csv("../../data/BR_env/forced_env.csv", index_col=["Unnamed: 0"])
raw_df = pd.DataFrame(raw_df, dtype="float")

In [3]:
INTERVAL = '10 min'
START_DATE = '2016-10-02 00:00'
END_DATE = raw_df.index[-1]
COMPRESSED_RANGE = pd.date_range(START_DATE, END_DATE, freq=INTERVAL)
TIME_STEPS = 144
OUTPUT_SIZE = 12

EXPLANATION = "Allenv_MT-LN-LSTM" "_T" + str(TIME_STEPS) + "_I" + INTERVAL

In [4]:
weather_df = raw_df[[col for col in raw_df.columns if '.w' in col]]
soil_df = raw_df[[col for col in raw_df.columns if 's_' in col]]
greenhouse_df = raw_df.drop([col for col in raw_df.columns if '.w' in col], axis=1)
greenhouse_df = greenhouse_df.drop([col for col in raw_df.columns if 's_' in col], axis=1)

In [5]:
new_coloumns = []
for col in greenhouse_df.columns:
    new_coloumns.append(col.split(".")[0])
greenhouse_df.columns = new_coloumns

new_coloumns = []
for col in soil_df.columns:
    new_coloumns.append(col.split(".")[0])
soil_df.columns = new_coloumns

In [6]:
greenhouse_df = greenhouse_df.groupby(greenhouse_df.columns, axis=1, sort=False).mean()
soil_df = soil_df.groupby(soil_df.columns, axis=1, sort=False).mean()

raw_df = pd.concat([greenhouse_df, soil_df, weather_df], axis=1)

In [7]:
raw_df.index = pd.DatetimeIndex(raw_df.index)
raw_df = raw_df.reindex(COMPRESSED_RANGE)

In [8]:
raw_label = raw_df[[col for col in list(greenhouse_df.columns) + list(soil_df.columns)]]
raw_label.index = pd.DatetimeIndex(raw_label.index)

In [9]:
raw_label = raw_label.drop(["ec", "atm", "uv"], axis=1)

In [10]:
raw_label

,temp,rh,co2,rad,s_temp,s_rh
2016-10-02 00:00:00,24.741111,79.428889,598.255556,0.222222,25.066667,30.450000
2016-10-02 00:10:00,24.741111,79.428889,598.255556,0.222222,25.066667,30.450000
2016-10-02 00:20:00,23.952222,82.173333,600.700000,0.111111,25.033333,30.450000
2016-10-02 00:30:00,23.641111,83.528889,605.033333,0.222222,25.033333,30.450000
2016-10-02 00:40:00,24.374444,82.117778,614.033333,0.222222,25.033333,30.450000
2016-10-02 00:50:00,25.607778,77.251111,609.477778,0.111111,25.033333,30.433333
2016-10-02 01:00:00,25.030000,78.595556,606.811111,0.111111,25.066667,30.433333
2016-10-02 01:10:00,24.263333,81.117778,609.700000,0.222222,25.016667,30.450000
2016-10-02 01:20:00,23.607778,83.462222,617.811111,0.111111,25.033333,30.450000
2016-10-02 01:30:00,23.885556,83.195556,622.477778,0.222222,25.000000,30.450000


In [10]:
MAXS = raw_df.max()
MINS = raw_df.min()
LABELMAXS = raw_label.max()
LABELMINS = raw_label.min()
raw_df = (raw_df - MINS) / (MAXS - MINS)
raw_label = (raw_label - LABELMINS) / (LABELMAXS - LABELMINS)

In [11]:
raw_input = raw_df.copy()
raw_input.index = pd.DatetimeIndex(raw_input.index)

In [12]:
TRAIN_RANGE = raw_label.index
TRAIN_RANGE = TRAIN_RANGE[TIME_STEPS:-OUTPUT_SIZE]

start_input = TRAIN_RANGE - pd.Timedelta(INTERVAL*(TIME_STEPS))
end_input = TRAIN_RANGE - pd.Timedelta(INTERVAL)
start_output = TRAIN_RANGE
end_output = TRAIN_RANGE + pd.Timedelta(INTERVAL*(OUTPUT_SIZE-1))

In [13]:
rnn_input = []
rnn_label = []

for i in range(len(TRAIN_RANGE)):
    input_range = pd.date_range(start_input[i], end_input[i], freq=INTERVAL)
    label_range = pd.date_range(start_output[i], end_output[i], freq=INTERVAL)
    
    rnn_input.append(raw_input.loc[input_range].values)
    rnn_label.append(raw_label.loc[label_range].values)
    
    
    if i % 10000 == 0:
        print("%6d/%6d" % (i, len(TRAIN_RANGE)))
    
print("%6d/%6d" % (i+1, len(TRAIN_RANGE)))
print("Process ended.")

     0/ 87252
 10000/ 87252
 20000/ 87252
 30000/ 87252
 40000/ 87252
 50000/ 87252
 60000/ 87252
 70000/ 87252
 80000/ 87252
 87252/ 87252
Process ended.


In [14]:
rnn_input = np.asarray(rnn_input)
rnn_label = np.asarray(rnn_label)

In [15]:
print("Input shape: ", end=""); print(rnn_input.shape)
print("Label shape: ", end=""); print(rnn_label.shape)
print("Type: %s" % type(rnn_input))

Input shape: (87252, 144, 15)
Label shape: (87252, 12, 6)
Type: <class 'numpy.ndarray'>


In [16]:
if not os.path.isdir("./data/"):
    os.makedirs("./data/")
f = open("./data/" + EXPLANATION + ".npz", 'wb')
np.savez(f,
         inputs = rnn_input, labels = rnn_label,
         mins = MINS, maxs = MAXS, labelmins = LABELMINS, labelmaxs = LABELMAXS,
         timesteps = TIME_STEPS, outputsize = OUTPUT_SIZE,
         start_input = start_input, start_output = start_output,
         end_input = end_input, end_output = end_output,
         interval = INTERVAL, explanation = EXPLANATION)
f.close()